<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/master/phrase_extraction_unterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/tr_ar


In [ ]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils


Cloning into 'code_utils'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 210 (delta 13), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (210/210), 739.83 KiB | 6.67 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [93]:
#For parsing functions
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer=nlp.tokenizer.tokens_from_list
print(dir(nlp.tokenizer))
def get_conll(sent_input):
  #nlp = spacy.load("en_core_web_sm")
  #if type(sent_input) is list: nlp.tokenizer=nlp.tokenizer.tokens_from_list
  doc = nlp(sent_input)
  conll_2d=[]
  for token in doc:	
    token_i = token.i+1
    if token.i==token.head.i: head_i=0
    else: head_i = token.head.i+1
    items=[token_i,token.text, token.lemma_, token.tag_, token.pos_, "_", head_i, token.dep_,"_","_"]
    conll_2d.append(items)
  return conll_2d

text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
text="she went to school"
words=text.split()
# for a in get_conll(text):
#   print(a)

for a in get_conll(words):
  print(a)

  


# doc = nlp(text)
# for d in doc:
#   print(d, d.head.i)
#dir(d, d.head)

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']
[1, 'she', '-PRON-', 'PRP', 'PRON', '_', 2, 'nsubj', '_', '_']
[2, 'went', 'go', 'VBD', 'VERB', '_', 0, 'ROOT', '_', '_']
[3, 'to', 'to', 'IN', 'ADP', '_', 2, 'prep', '_', '_']
[4, 'school', 'school', 'NN', 'NOUN', '_', 3, 'pobj', '_', '_']


In [ ]:
#Reading UNTerm entries from the TSV files with tokenized entries, creating a list
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter

fopen=open("unterm_tokenized.tsv")

all_en_words=[]
unterm_list=[]
pivot_fwd_index=[]
for i,line in enumerate(fopen):
  item=line.strip("\n\r\t").split("\t")
  en_words_str,ar_words_str=item
  en_words,ar_words=en_words_str.split(" "),ar_words_str.split(" ")
  unterm_list.append((en_words,ar_words))
  en_words_lower=[v.lower() for v in en_words]
  #en_word_count=sorted([(v,en_counter[v]) for v in en_words_lower],key=lambda x:x[1])
  all_en_words.extend(en_words_lower)
en_counter=Counter(all_en_words)

#Now creating the pivot dictionary for the English, to allow easy retrieval of the terms from any text
for i,item in enumerate(unterm_list):
  en_words,ar_words=item  
  en_words_lower=[v.lower() for v in en_words]
  en_word_count=sorted([(v,en_counter[v]) for v in en_words_lower],key=lambda x:x[1])
  #print(">>>", en_word_count)
  pivot_word=en_word_count[0][0]
  pivot_fwd_index.append((pivot_word,i))
  # print("pivot:",pivot_word)
  # print(en_words_lower)
  # print(ar_words)
  # print("----")
pivot_fwd_index.sort()
#print(pivot_fwd_index)
pivot_dict=dict(iter([(key,[v[1] for v in list(group)]) for key,group in groupby(pivot_fwd_index,lambda x:x[0])]))
print("finished preparing pivot dictionary")
#print(pivot_dict)
fopen.close()
print(len(unterm_list))
print(unterm_list[10])

finished preparing pivot dictionary
81322
(['European', 'Convention', 'on', 'the', 'Compensation', 'of', 'Victims', 'of', 'Violent', 'Crimes'], ['ال_اتفاقية', 'ال_أوروبية', 'ال_متعلقة', 'بـ', 'تعويض', 'ضحايا', 'جرائم', 'ال_عنف'])


In [ ]:
#processing UNTerm entries - we should do this as the first step when we get a new version of UNTerm
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter


ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})
print("processed count dictionary")
#ar_pre_suf(ar_word,cur_ar_counter_dict={})
unterm_open=open("all_unterm.tsv")
unterm_list=[]
all_en_words=[]
for i,f in enumerate(unterm_open):
  if i==0: continue
  if i%5000==0: print(i)

  f_split=f.strip("\n\r\t").split("\t")
  if len(f_split)<5: continue
  # if i<1000: continue
  # if i>1300: break
  en=f_split[1]
  ar=clean_ar(f_split[4])
  
  en_words,ar_words_tmp=tok(en),tok(ar)
  if not en_words: continue
  if not ar_words_tmp: continue
  en_lower=[v.lower() for v in en_words]
  all_en_words.extend(en_lower)

  ar_words=tok_ar(ar_words_tmp,ar_counter_dict)
  unterm_list.append((en_words,ar_words))
  # print(en_words)
  # #print(ar_words_tmp)
  # print(ar_words)  
  # print("---------")
  # for ar_w in ar_words:
  #   print(ar_w,ar_counter_dict.get(ar_w))
  #   cur_candidates=get_pre_suf_candidates(ar_w)
  #   sorted_candidates=sort_filter(cur_candidates,ar_counter_dict)
  #   for s in sorted_candidates:
  #     print(">>>", s)

  #   # candidates=ar_pre_suf(ar_w,ar_counter_dict)
  #   # print(ar_w)
  #   # print(candidates[0])
unterm_open.close()
print("finished processing %s terms"%len(unterm_list))
en_counter=Counter(all_en_words)

#preparing a tsv with tokenized UN Term items
fopen=open("unterm_tokenized.tsv","w")
for en0,ar0 in unterm_list:
  en_str=" ".join(en0)
  ar_str=" ".join(ar0)
  line="%s\t%s\n"%(en_str,ar_str)
  fopen.write(line)
fopen.close()

processed count dictionary
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
finished processing 81322 terms


# Working with Bitext Corpus
In the following section, we are working with the UN Bitext Corpus

In [47]:
#Now processing the corpus
#loading the Arabic counter to do the tokenization of Arabic text
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter

ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents_tok,trg_sents_tok=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=250000: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  en_toks,ar_toks=tok(en_line),tok(ar_line)
  ar_toks=tok_ar(ar_toks,ar_counter_dict)
  src_sents_tok.append(en_toks)
  trg_sents_tok.append(ar_toks)
  # print(en_toks)
  # print(ar_toks)
  # print("----")
ar_fopen.close()
en_fopen.close()
print(len(src_sents_tok))

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
250000


In [48]:
import math
#The functions used for indexing a pair of lists of tokenized sentences
def norm_tok(token): #a simple function to lowercase and remove alif laam, can include other factors
  token=token.lower()
  if "_" in token: token=token.split("_")[1]
  return token
import unicodedata
def is_punct(token):
  if token=="": return True
  return unicodedata.category(token[0])[0]=="P"
def rtrv_idx(tokens,idx_dict): #get the indexes of a phrase - we can extend it to as many tokens as we want
    cur_indexes=idx_dict.get(tokens[0],[])
    for ti,ta in enumerate(tokens[1:]):
        cur_offset=ti+1
        cur_raw_idx=idx_dict.get(ta,[])
        cur_offset_idx=[(v[0],v[1]-cur_offset) for v in cur_raw_idx]
        cur_indexes=list(set(cur_indexes).intersection(set(cur_offset_idx)))
    cur_indexes=[v[0] for v in cur_indexes]
    cur_indexes=sorted(list(set(cur_indexes)))
    return cur_indexes

def idx_match(idx1,idx2): #This function is to give a ratio of matching between the inverted indexes for two words
    if len(idx1)==0 or len(idx2)==0: return 0
    intersection=len(list(set(idx1).intersection(set(idx2))))
    union=len(idx1)+len(idx2)-intersection
    if union==0: return 0
    return float(intersection)/union

def get_longest_match(sent_tokens,idx_dict,min_n_indexes=5,max_phrase_size=7):
  cur_matching_list=[]
  for tok_i,cur_token in enumerate(sent_tokens):
    cur_seq=sent_tokens[tok_i:]
    cur_indexes=idx_dict.get(cur_seq[0],[])
    tmp_indexes=[v[0] for v in cur_indexes]
    tmp_indexes=sorted(list(set(tmp_indexes)))
    cur_matching_list.append(([cur_token],(tok_i,tok_i),tmp_indexes))

    for ti,ta in enumerate(cur_seq[1:]):
        cur_offset=ti+1
        cur_phrase=cur_seq[:cur_offset+1]
        if len(cur_phrase)>max_phrase_size: break
        cur_raw_idx=idx_dict.get(ta,[])
        cur_offset_idx=[(v[0],v[1]-cur_offset) for v in cur_raw_idx]
        cur_indexes=list(set(cur_indexes).intersection(set(cur_offset_idx)))
        

        if len(cur_indexes)<min_n_indexes: break
        tmp_indexes=[v[0] for v in cur_indexes]
        tmp_indexes=sorted(list(set(tmp_indexes)))
        cur_matching_list.append((cur_phrase,(tok_i,tok_i+cur_offset),tmp_indexes))
  return cur_matching_list



class walign:
  def __init__(self,src_sents,trg_sents,ignore_punct=True,ignore_ar_pre_suf=False,ignored_src_toks=[],ignored_trg_toks=[]):
    self.src_sents=src_sents
    self.trg_sents=trg_sents
    self.ignore_punct=ignore_punct
    self.ignore_ar_pre_suf=ignore_ar_pre_suf
    self.ignored_src_toks=ignored_src_toks
    self.ignored_trg_toks=ignored_trg_toks
    self.src_fwd_index,self.trg_fwd_index=[],[]
    for s_i,st in enumerate(zip(self.src_sents,self.trg_sents)): 
      if s_i%20000==0: print("indexing %s"%s_i)
      src_toks,trg_toks=st
      src_toks=[norm_tok(v) for v in src_toks]
      trg_toks=[norm_tok(v) for v in trg_toks]

      if self.ignore_punct:
        src_toks=[v for v in src_toks if not is_punct(v) and not v in self.ignored_src_toks]
        trg_toks=[v for v in trg_toks if not is_punct(v) and not v in self.ignored_trg_toks]
      else:
        src_toks=[v for v in src_toks if not v in self.ignored_src_toks]
        trg_toks=[v for v in trg_toks if not v in self.ignored_trg_toks]


      src_toks=[(v,s_i,t_i) for t_i,v in enumerate(src_toks)]
      trg_toks=[(v,s_i,t_i) for t_i,v in enumerate(trg_toks)]
      self.src_fwd_index.extend(src_toks)
      self.trg_fwd_index.extend(trg_toks)
    self.src_fwd_index.sort()
    self.trg_fwd_index.sort()
    src_grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(self.src_fwd_index,lambda x:x[0])]
    trg_grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(self.trg_fwd_index,lambda x:x[0])]

    self.src_inverted=dict(iter(src_grouped))
    self.trg_inverted=dict(iter(trg_grouped))  
    print("finished indexing %s pairs of sentences"%len(self.src_sents))
  def align_sent_words(self):
    phrase_list_fname="test_phrases12.tsv"
    test_fopen=open(phrase_list_fname,"w")
    test_fopen.close()  
    test_i=0
    n_sents=10000
    min_ratio=0.1
    for s_i,st in enumerate(zip(self.src_sents,self.trg_sents)):
      #if s_i>100: break 
      if s_i<test_i: continue
      if s_i>test_i+n_sents: break
      src_toks,trg_toks=st
      src_toks_orig,trg_toks_orig=st
      if s_i%10==0: print("sentence #", s_i)
      # print("sentence #", s_i)
      # print("original src:",src_toks)
      # print("original trg:",trg_toks)
      src_toks=[norm_tok(v) for v in src_toks]
      trg_toks=[norm_tok(v) for v in trg_toks]

      if self.ignore_punct:
        src_toks=[(v,i_) for i_,v in enumerate(src_toks) if not is_punct(v) and not v in self.ignored_src_toks]
        trg_toks=[(v,i_) for i_,v in enumerate(trg_toks) if not is_punct(v) and not v in self.ignored_trg_toks]
      else:
        src_toks=[(v,i_) for i_,v in enumerate(src_toks) if not v in self.ignored_src_toks]
        trg_toks=[(v,i_) for i_,v in enumerate(trg_toks) if not v in self.ignored_trg_toks]

      src_tok_map=[v[1] for v in src_toks] #mapping the new token sequence (after removing punctuation and stopwords) to the old sequence
      trg_tok_map=[v[1] for v in trg_toks]
      src_toks=[v[0] for v in src_toks]
      trg_toks=[v[0] for v in trg_toks]
      # print("filtered src:",src_toks,src_tok_map)
      # print("filtered trg:",trg_toks,trg_tok_map)
      # print("------")

      tmp_src_dict=dict(iter([(v,self.src_inverted[v]) for v in src_toks]))
      tmp_trg_dict=dict(iter([(v,self.trg_inverted[v]) for v in trg_toks]))

      src_phrases_indexes=get_longest_match(src_toks,tmp_src_dict,min_n_indexes=5,max_phrase_size=4)
      trg_phrases_indexes=get_longest_match(trg_toks,tmp_trg_dict,min_n_indexes=5,max_phrase_size=4)
      matching_phrases_list=[]
      for a in src_phrases_indexes:
        src_phrase,src_locs,src_indexes=a
        if len(src_indexes)<10: continue
        for b in trg_phrases_indexes:
          trg_phrase,trg_locs,trg_indexes=b
          if len(trg_indexes)<10: continue
          ratio=idx_match(src_indexes,trg_indexes)
          if ratio<min_ratio: continue
          #ratio+=0.001*(len(src_phrase)+len(trg_phrase))
          combined_src_trg_len=len(src_phrase)+len(trg_phrase)+20 #arbitrary number to make the log difference small
          n_src_trg_indexes=len(src_indexes)+len(trg_indexes)
          wt=ratio*math.log(n_src_trg_indexes)*math.log(combined_src_trg_len)
          matching_phrases_list.append((src_phrase,trg_phrase,src_locs,trg_locs,n_src_trg_indexes,ratio,wt))
      matching_phrases_list.sort(key=lambda x:-x[-1])
      used_src,used_trg=[],[]
      test_fopen=open(phrase_list_fname,"a")
      copy_list=sorted(matching_phrases_list,key=lambda x:x[0])
      grouped_copy=[(key,[v for v in list(group)]) for key,group in groupby(copy_list,lambda x:x[0])]
      for k, grp in grouped_copy:
        grp.sort(key=lambda x:-x[-1])
        for g in grp[:3]:
          sp0,tp0,sl0,tl0,ni,r0,wt0=g
          si_0,si_1=src_tok_map[sl0[0]],src_tok_map[sl0[1]]
          ti_0,ti_1=trg_tok_map[tl0[0]],trg_tok_map[tl0[1]]
          orig_src_phrase=" ".join([v.lower() for v in src_toks_orig[si_0:si_1+1]]) 
          orig_trg_phrase=" ".join([v.lower() for v in trg_toks_orig[ti_0:ti_1+1]]) 

          #print(m)
          #print(" ".join(orig_src_phrase))
          #print(orig_src_phrase, orig_trg_phrase, r0)
          line="%s\t%s\t%s\t%s\t%s\t%s\n"%(s_i,orig_src_phrase, orig_trg_phrase,ni, r0,wt0)

          test_fopen.write(line)


        # print(k)
        # print(grp[:2])
        # print("------")
      for m in matching_phrases_list:
        #continue
        sp0,tp0,sl0,tl0,ni,r0,wt0=m
        if sp0 in used_src: continue
        if tp0 in used_trg: continue
        used_src.append(sp0)
        used_trg.append(tp0)
        si_0,si_1=src_tok_map[sl0[0]],src_tok_map[sl0[1]]
        ti_0,ti_1=trg_tok_map[tl0[0]],trg_tok_map[tl0[1]]
        orig_src_phrase=" ".join([v.lower() for v in src_toks_orig[si_0:si_1+1]]) 
        orig_trg_phrase=" ".join([v.lower() for v in trg_toks_orig[ti_0:ti_1+1]]) 

        #print(m)
        #print(" ".join(orig_src_phrase))
        #print(orig_src_phrase, orig_trg_phrase, r0)
        line="%s\t%s\t%s\t%s\t%s\t%s\n"%(s_i,orig_src_phrase, orig_trg_phrase,ni, r0,wt0)

        test_fopen.write(line)
      test_fopen.close()  
      #print("------")
            

src_stopwords=["the","and","of","an","a","in","at","on","be","is","are","was","were","to","has","with","for","by","it","its","has","have","been"]
trg_stopwords=["وـ","لـ","بـ","في","على","ـه","ـها","إلى"] #we should also include ignoring prefixes and suffixes with tatweel

walign_obj=walign(src_sents_tok,trg_sents_tok,ignored_src_toks=src_stopwords, ignored_trg_toks=trg_stopwords)
#Doing the actual word alignment
#walign_obj.align_sent_words()


indexing 0
indexing 20000
indexing 40000
indexing 60000
indexing 80000
indexing 100000
indexing 120000
indexing 140000
indexing 160000
indexing 180000
indexing 200000
indexing 220000
indexing 240000
finished indexing 250000 pairs of sentences


In [ ]:
import time

def get_span_phrase(span_input,src_toks_input,trg_toks_input):
  (span1_x0,span1_x1),(span1_y0,span1_y1)=span_input
  return src_toks_input[span1_x0:span1_x1+1],trg_toks_input[span1_y0:span1_y1+1]
def combine_spans(span1,span2):
  (span1_x0,span1_x1),(span1_y0,span1_y1)=span1
  (span2_x0,span2_x1),(span2_y0,span2_y1)=span2
  new_span_xs=min(span1_x0,span2_x0),max(span1_x1,span2_x1)
  new_span_ys=min(span1_y0,span2_y0),max(span1_y1,span2_y1)
  return new_span_xs,new_span_ys

def spans_contiguous(span1,span2):
  (span1_x0,span1_x1),(span1_y0,span1_y1)=span1
  (span2_x0,span2_x1),(span2_y0,span2_y1)=span2
  if (span1_x0,span1_x1)==(span2_x0,span2_x1) and (span2_y0==span1_y1+1 or span1_y0==span2_y1+1): return True
  if (span1_y0,span1_y1)==(span2_y0,span2_y1) and (span2_x0==span1_x1+1 or span1_x0==span2_x1+1): return True
  return False


def test_align(s_i):
  min_ratio=0.05 #min ratio of matching between src and trg indexes
  t0=time.time()
  #first we adjust and normalize the sentence pair tokens
  src_toks,trg_toks=walign_obj.src_sents[s_i],walign_obj.trg_sents[s_i]
  src_toks=["<s>"]+src_toks+["</s>"] #add sentence start and end markers
  trg_toks=["<s>"]+trg_toks+["</s>"]

  src_toks_orig,trg_toks_orig=src_toks,trg_toks #keep track of the original sentences without the following processing
  #if s_i%10==0: print("sentence #", s_i) 
  # print("sentence #", s_i)
  # print("original src:",src_toks)
  # print("original trg:",trg_toks)
  src_toks=[norm_tok(v) for v in src_toks] #normalize sentences to match indexes (lowercase, remove alif ... etc)
  trg_toks=[norm_tok(v) for v in trg_toks]


  if walign_obj.ignore_punct: #exclude punctuation and stop words in src and trg
    src_toks=[(v,i_) for i_,v in enumerate(src_toks) if not is_punct(v) and not v in walign_obj.ignored_src_toks]
    trg_toks=[(v,i_) for i_,v in enumerate(trg_toks) if not is_punct(v) and not v in walign_obj.ignored_trg_toks]
  else:
    src_toks=[(v,i_) for i_,v in enumerate(src_toks) if not v in walign_obj.ignored_src_toks]
    trg_toks=[(v,i_) for i_,v in enumerate(trg_toks) if not v in walign_obj.ignored_trg_toks]


  # print(src_toks)
  # print(trg_toks)
  src_tok_map=[v[1] for v in src_toks] #mapping the new token sequence (after removing punctuation and stopwords) to the old sequence
  trg_tok_map=[v[1] for v in trg_toks]
  src_toks=[v[0] for v in src_toks]
  trg_toks=[v[0] for v in trg_toks]
  # print("filtered src:",src_toks,src_tok_map)
  # print("filtered trg:",trg_toks,trg_tok_map)
  # print("------")
  #now sentences are normalized to start working ith word indexes
  

  
  #we create temporary inverted indexes for the current sentence pair words
  tmp_src_dict=dict(iter([(v,walign_obj.src_inverted.get(v,[])) for v in src_toks]))
  tmp_trg_dict=dict(iter([(v,walign_obj.trg_inverted.get(v,[])) for v in trg_toks]))
  t1=time.time()
  #print("started processing indexes", t1-t0)
  t0=time.time()

  #now we start using the indexes to get the longest matching token sequence in src and trg
  src_phrases_indexes=get_longest_match(src_toks,tmp_src_dict,min_n_indexes=50,max_phrase_size=4)
  trg_phrases_indexes=get_longest_match(trg_toks,tmp_trg_dict,min_n_indexes=50,max_phrase_size=4)
  t1=time.time()
  #print("obtained longest matches", t1-t0)
  t0=time.time()

  #now we compare all the identified token sequences in src and trg
  matching_phrases_list=[]
  for a in src_phrases_indexes:
    src_phrase,src_locs,src_indexes=a #the longest match function returns the phrase, its locations, and its indexes
    if len(src_indexes)<10: continue #we can add some restriction here to exclude phrases with few indexes, but I think it is already included in the longest match functio
    for b in trg_phrases_indexes:
      trg_phrase,trg_locs,trg_indexes=b
      if len(trg_indexes)<10: continue
      ratio=idx_match(src_indexes,trg_indexes) #we get a ratio of matching (intersection(src_indexes,trg_indexes)/union(src_indexes,trg_indexes))
      if ratio<min_ratio: continue
      #ratio+=0.001*(len(src_phrase)+len(trg_phrase))
      
      combined_src_trg_len=len(src_phrase)+len(trg_phrase)+10 #arbitrary number to make the log difference small
      #ratio=ratio*math.log(combined_src_trg_len)

      n_src_trg_indexes=len(src_indexes)+len(trg_indexes)
      wt=ratio*math.log(n_src_trg_indexes)*math.log(combined_src_trg_len) #trying to give more weight to more frequent phrase pairs and larger phrases
      #print(n_src_trg_indexes, math.log(n_src_trg_indexes), combined_src_trg_len, math.log(combined_src_trg_len), ratio, wt)
      matching_phrases_list.append((src_phrase,trg_phrase,src_locs,trg_locs,n_src_trg_indexes,ratio,wt))
  #matching_phrases_list.append(("<s>","<s>",[0,0],[0,0],1,1,50))
  #matching_phrases_list.append(("</s>","</s>",[len(src_toks)-1,len(src_toks)-1],[len(trg_toks)-1,len(trg_toks)-1],1,1,50))
  #matching_phrases_list.append((src_phrase,trg_phrase,src_locs,trg_locs,n_src_trg_indexes,ratio,wt))
  t1=time.time()
  #print("obtained phrase index matching list", t1-t0)
  t0=time.time()


  matching_phrases_list.sort(key=lambda x:-x[-1])
  matching_phrases_list.sort(key=lambda x:x[2]) #sort the phrase pairs by ratio, then group them by src phrase
  grouped_matching=[(key,[v for v in list(group)]) for key,group in groupby(matching_phrases_list,lambda x:x[0])]

  span_dict={} #populate a span dict, showing the weight (combined ratio) of each span
  start_span=((0,0),(0,0)) #NW span
  end_span=((len(src_toks)-1,len(src_toks)-1),(len(trg_toks)-1,len(trg_toks)-1)) #SE span
  right_span=((len(src_toks)-1,len(src_toks)-1),(0,0)) #NE span - to create paths going upwards to the right
  span_dict[start_span]=1
  span_dict[end_span]=1
  span_dict[right_span]=1
  for k, grp in grouped_matching: #now we work with each src phrase, with its corresponding trg phrase and respective ratio
    #print(k)
    grp.sort(key=lambda x:-x[-1])
    for g in grp[:10]:
      cur_span=g[2:4]
      cur_ratio=g[-2]
      cur_wt=g[-1]
      span_dict[cur_span]=cur_ratio #populate span dict with these values
      #span_dict[cur_span]=cur_wt
    #   print(g[2:4])
    #   print(g[:2],g[-2])
    # print("---")
  # print(span_dict)
  sorted_spans=sorted(span_dict.keys())
  #using these spans, we will try finding the shortest path from each span to the NE end point
  #to account for situations where the sequence of words is not linear SE e.g. General assembly = الجمعية العامة
  #we implement a shortest path algorithm first twards NE, then towards SE
  pre_path_dict={}
  pre_prev_dict={}
  #transition_dict={}
  for span_i,span1 in enumerate(sorted_spans): 
    # span1_xs,span1_ys=span1
    # span1_x0,span1_x1=span1_xs
    # span1_y0,span1_y1=span1_ys
    (span1_x0,span1_x1),(span1_y0,span1_y1)=span1
    prev_wt=0
    prev_span=pre_prev_dict.get(span1)
    if prev_span!=None: prev_wt=pre_path_dict.get(prev_span,0)
    cur_wt=span_dict[span1]+prev_wt
    pre_path_dict[span1]=cur_wt
    for span2 in sorted_spans:
      if span2==span1: continue
      # span2_xs,span2_ys=span2
      # span2_x0,span2_x1=span2_xs
      # span2_y0,span2_y1=span2_ys
      (span2_x0,span2_x1),(span2_y0,span2_y1)=span2
      if span2_x0<=span1_x1: continue
      if span2_y1>=span1_y0: continue
      #print(">>>>>>>>>>>>>>>> span1",span1,"span2",span2)
      next_wt=span_dict[span2]
      found_wt=pre_path_dict.get(span2,0)
      combined_wt=cur_wt+next_wt
      if combined_wt>found_wt:
        pre_path_dict[span2]=combined_wt
        pre_prev_dict[span2]=span1
        #pre_prev_dict[span1]=span2
  # for key,val in pre_prev_dict.items():
  #   print("pre-populated prev_dict", key,val)
  for cur_span in sorted_spans:
    #print(">>>>", span1,pre_prev_dict.get(span1))
    pre_final_spans=[]
    our_pt=pre_prev_dict.get(cur_span)
    if our_pt: 
      pre_final_spans.append(cur_span)
      pre_final_spans.append(our_pt)
    while our_pt: #identify the shortest NE path from this point
      our_pt=pre_prev_dict.get(our_pt)
      #print("our_pt",our_pt)
      if our_pt: pre_final_spans.append(our_pt)
    if pre_final_spans: 
      pre_final_spans.reverse()
      for i0 in range(len(pre_final_spans)-1):
        for i1 in range(i0+2,len(pre_final_spans)):
          cur_slice=pre_final_spans[i0:i1] #we slice the path to get NE rectangles 
          cur_slice_wt=sum([span_dict[v] for v in cur_slice])
          first_span,last_span=cur_slice[0],cur_slice[-1]
          (span1_x0,span1_x1),(span1_y0,span1_y1)=first_span
          (span2_x0,span2_x1),(span2_y0,span2_y1)=last_span
          new_span=(span1_x0,span2_x1),(min(span1_y0,span2_y0),max(span1_y1,span2_y1))
          span_dict[new_span]=cur_slice_wt #we add the NE rectangle to the span dict, to use in the SE iteration
      #     print(i0,i1, cur_slice_wt,new_span)
      #     print("cur_slice",cur_slice)
      #     print("new_span",new_span)
      # print("pre final upward paths>>>>", pre_final_spans)

  # return 


  path_dict={}
  prev_dict={}
  #transition_dict={}
  for span_i,span1 in enumerate(sorted_spans): #now we find the shortest path from the NW point to SE point 
    # span1_xs,span1_ys=span1
    # span1_x0,span1_x1=span1_xs
    # span1_y0,span1_y1=span1_ys
    (span1_x0,span1_x1),(span1_y0,span1_y1)=span1
    prev_wt=0
    prev_span=prev_dict.get(span1)
    if prev_span!=None: prev_wt=path_dict.get(prev_span,0)
    cur_wt=span_dict[span1]+prev_wt
    path_dict[span1]=cur_wt
    for span2 in sorted_spans:
      if span2==span1: continue
      # span2_xs,span2_ys=span2
      # span2_x0,span2_x1=span2_xs
      # span2_y0,span2_y1=span2_ys
      (span2_x0,span2_x1),(span2_y0,span2_y1)=span2
      # if span1[0]==span2[0] and span2_y0==span1_y1+1: pass
      # elif span1[1]==span2[1] and span2_x0==span1_x1+1: pass
      if span2_x0<=span1_x1: continue
      if span2_y0<=span1_y1: continue
      next_wt=span_dict[span2]
      found_wt=path_dict.get(span2,0)
      combined_wt=cur_wt+next_wt
      if combined_wt>found_wt:
        path_dict[span2]=combined_wt
        prev_dict[span2]=span1
      # transition_dict[(span1,span2)]=next_wt

      # print(span1)
      # print(span2)
      # print("---")
  final_spans=[]
  our_pt=prev_dict[end_span] #here is where we get the shortest path from the end span (SE) backwards
  final_spans.append(our_pt)
  while our_pt:
    our_pt=prev_dict.get(our_pt)
    if our_pt: final_spans.append(our_pt) #and this is our list of spans making the shortest path

  final_spans.reverse()
  used_xs,used_ys=[],[]
  src_locs_dict={}
  trg_locs_dict={}
  for span0 in final_spans: #we now extract information about the final spans, mainly used x and y coordinates
    span0_xs,span0_ys=span0
    src_locs_dict[span0_xs]=span0#easy identifier for which span corresponds to a certain x-range
    trg_locs_dict[span0_ys]=span0#and certain y-range
    span0_x0,span0_x1=span0_xs
    span0_y0,span0_y1=span0_ys
    used_xs.extend(range(span0_x0,span0_x1+1))
    used_ys.extend(range(span0_y0,span0_y1+1))
  #matching_phrases_list.append((src_phrase,trg_phrase,src_locs,trg_locs,n_src_trg_indexes,ratio,wt))
  ######################################
  ######################################
  
  #now we try to collect the spans that have gaps (not identified on the shortest path)
  matching_phrases_list.sort(key=lambda x:-x[-2])
  for m in matching_phrases_list:
    src_phrase,trg_phrase,src_locs,trg_locs,n_src_trg_indexes,ratio,wt=m
    cur_span=(src_locs,trg_locs)
    cur_ratio=ratio
    corr_src_span=src_locs_dict.get(src_locs)
    corr_trg_span=trg_locs_dict.get(trg_locs)
    if cur_span==corr_src_span and cur_span==corr_trg_span: continue
    #print("cur_span",cur_span, span_dict[cur_span], get_span_phrase(cur_span,src_toks,trg_toks))
    check_not_used_ys=True
    if any([v in used_ys for v in range(trg_locs[0],trg_locs[1]+1)]): check_not_used_ys=False
    check_not_used_xs=True
    if any([v in used_xs for v in range(src_locs[0],src_locs[1]+1)]): check_not_used_xs=False      
    
    if corr_src_span: #if the current span shares the same x-range as one of the final spans
      is_contiguous=spans_contiguous(cur_span,corr_src_span)
      if is_contiguous and check_not_used_ys:
        # print("cur_span",cur_span, span_dict[cur_span], get_span_phrase(cur_span,src_toks,trg_toks))
        # print("corr_src_span",corr_src_span,span_dict[corr_src_span])
        # print(get_span_phrase(corr_src_span,src_toks,trg_toks), is_contiguous,"check_not_used",check_not_used_ys)
        
        y_range_to_add_to_used=list(range(trg_locs[0],trg_locs[1]+1))
        used_ys.extend(y_range_to_add_to_used)
        #print("y_range_to_add_to_used", y_range_to_add_to_used)
        new_span=combine_spans(cur_span,corr_src_span)
        final_spans.append(new_span)
        span_dict[new_span]=cur_ratio+span_dict[corr_src_span]
        if corr_src_span in final_spans: final_spans.remove(corr_src_span)
        #final_spans
        # print(new_span,get_span_phrase(new_span,src_toks,trg_toks))
        # print("-----------")
        
    
    if corr_trg_span:  #if the current span shares the same x-range as one of the final spans
      #if any([v in used_xs for v in range(src_locs[0],src_locs[1]+1)]): check_not_used=False      
      is_contiguous=spans_contiguous(cur_span,corr_trg_span)
      if is_contiguous and check_not_used_xs:
        # print("cur_span",cur_span, span_dict[cur_span], get_span_phrase(cur_span,src_toks,trg_toks))
        # print("corr_trg_span",corr_trg_span,span_dict[corr_trg_span])
        # print(get_span_phrase(corr_trg_span,src_toks,trg_toks), is_contiguous,"check_not_used",check_not_used_xs)
        x_range_to_add_to_used=list(range(src_locs[0],src_locs[1]+1))
        used_xs.extend(x_range_to_add_to_used)
        #print("x_range_to_add_to_used", x_range_to_add_to_used)
        new_span=combine_spans(cur_span,corr_trg_span)
        final_spans.append(new_span)
        span_dict[new_span]=cur_ratio+span_dict[corr_trg_span]
        #print("final_spans",final_spans,"corr_trg_span",corr_trg_span)
        if corr_trg_span in final_spans: final_spans.remove(corr_trg_span)

        # print(new_span,get_span_phrase(new_span,src_toks,trg_toks))
        # print("-----------")
    if check_not_used_xs and check_not_used_ys: #if both x-coordinates and y-coordinates of the current span were not used before
       #print(">>>>>> cur_span",cur_span, span_dict[cur_span], get_span_phrase(cur_span,src_toks,trg_toks))
       y_range_to_add_to_used=list(range(trg_locs[0],trg_locs[1]+1))
       used_ys.extend(y_range_to_add_to_used)
       x_range_to_add_to_used=list(range(src_locs[0],src_locs[1]+1))
       used_xs.extend(x_range_to_add_to_used) 
       final_spans.append(cur_span)

    #print("src_locs,trg_locs",src_locs,trg_locs,corr_src_span,corr_trg_span)



  list_phrase_pairs=[]
  for span0 in final_spans: #final step to get all the spans and corresponding original phrase pairs
    span0_xs,span0_ys=span0
    span0_x0,span0_x1=span0_xs
    span0_y0,span0_y1=span0_ys
    #phrase_pair_wt=span_dict[span0]
    phrase_pair_wt=span_dict.get(span0,0) #we need to investigate why there can be an issue with missing a span
    #print(span_dict[span0])

    # x0_ex_list,x1_ex_list=[],[] #extended x & y points outside the current span
    # x0_ex_list,x1_ex_list=[],[] #which are not used by other span
    # for i_ in range(1,4):
    #   if span0_x0-i_ in used_xs or span0_x0-i_<0: break
    #   x0_ex_list.append(span0_x0-i_)
    # for i_ in range(1,4):
    #   if span0_x1+i_ in used_xs or span0_x1+i_>=len(src_toks): break
    #   x1_ex_list.append(span0_x1+i_)
    # print("span0",span0,"x0_ex_list",x0_ex_list,"x1_ex_list",x1_ex_list)

    x0_,x1_=(src_tok_map[span0_xs[0]],src_tok_map[span0_xs[1]])
    y0_,y1_=(trg_tok_map[span0_ys[0]],trg_tok_map[span0_ys[1]])

    # x0_,x1_=(src_tok_map[span0_xs[0]],src_tok_map[span0_xs[1]])
    # y0_,y1_=(trg_tok_map[span0_ys[0]],trg_tok_map[span0_ys[1]])
    #print(span0_xs, (x0_,x1_), src_toks[span0_x0:span0_x1+1],src_toks_orig[x0_:x1_+1])
    #print(span0_ys, (y0_,y1_), trg_toks[span0_y0:span0_y1+1],trg_toks_orig[y0_:y1_+1])
    src_phrase=" ".join(src_toks_orig[x0_:x1_+1])
    trg_phrase=" ".join(trg_toks_orig[y0_:y1_+1])
    if src_phrase=="<s>": continue
    if src_phrase==trg_phrase: continue
    list_phrase_pairs.append((src_phrase,trg_phrase,phrase_pair_wt))
    #print(src_phrase,trg_phrase,phrase_pair_wt)


    
    #print("--------")
  return list_phrase_pairs

# s_i=0
# test_align(s_i)
fname="test_phrase_pairs4.tsv"
# fopen=open(fname,"w")
# fopen.close()

n_sents=230000
start_i=20000
start_i=42860
end_i=250000
#end_i=start_i+n_sents

for s_i in range(start_i,end_i):
  #print(s_i)
  if s_i%100==0: print(s_i)
  pairs=test_align(s_i)
  for p in pairs:
    #print(p)
    fopen=open(fname,"a")
    line="%s\t%s\t%s\t%s\n"%(s_i,p[0],p[1],round(p[2],4))
    fopen.write(line)
    fopen.close()
  #   print(line)
  # print("=============")


20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
3660

# QA section

In [28]:
fname="phrase_pairs_qa.tsv"
src_counter={}
trg_counter={}
corr_dict={}
src_trg_wt_dict={}

fopen=open(fname)
for i_,line in enumerate(fopen):
  split=line.strip("\n").split("\t")
  if len(split)!=4: continue
  sent_i,src,trg,wt=split
  src=src.lower()
  wt=float(wt)
  if wt==0: continue
  src_counter[src]=src_counter.get(src,0)+1
  trg_counter[trg]=src_counter.get(trg,0)+1
  src_trg_wt_dict[(src,trg)]=wt
  tmp_dict=corr_dict.get(src,{})
  tmp_dict[trg]=tmp_dict.get(trg,0)+1
  corr_dict[src]=tmp_dict




  #print(i_,split)
  #if i_>20000: break
fopen.close()

print(len(corr_dict.keys()))
src_word_index_dict={}
for src0 in corr_dict:
  src0_words=src0.split(" ")
  first=src0_words[0]
  src_word_index_dict[first]=src_word_index_dict.get(first,[])+[src0]

#print(src_word_index_dict["united"])
test_word="approved"
for a in src_word_index_dict[test_word]:
  print(a,corr_dict[a])



# for key,val in corr_dict.items():
#   print(key,val)

12942
approved {'ال_معتمدة': 23, 'وافق': 41, 'ال_موافق': 11, 'وفق': 8, 'وافقت': 53, 'ال_موافقة': 14, 'موافقة': 1, 'معتمدة': 1}
approved by the general {'وافقت علي': 5}


In [101]:
#Now the QA part
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter
from math import log
import random
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer=nlp.tokenizer.tokens_from_list

ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def random_color():
  rand = lambda: random.randint(150, 255)
  return '#%02X%02X%02X' % (rand(), rand(), rand())
        
        

output_fname="qa_out1.html"
output_fopen=open(output_fname,"w")
header='''
<html>
<head>
  <title>Bootstrap Example</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</head>
<body><table class="table table-bordered table-striped">
'''
output_fopen.write(header)
#output_fopen.write('<html><body><table class="table" border="1">')
fname="doc_bitext_qa.tsv"
fopen=open(fname)
for i_,line in enumerate(fopen):
  split=line.strip("\n").split("\t")
  if len(split)!=2: continue
  src,trg=split
  if src==trg: continue
  if not trg: continue
  #src=src.lower()
  trg=clean_ar(trg)
  en_toks,ar_toks=tok(src),tok(trg)
  ar_toks=tok_ar(ar_toks,ar_counter_dict)

  en_conll=get_conll(en_toks)
  heads=[i0_ for i0_,v in enumerate(en_conll) if v[6]==0]
  print("heads >>>>>",heads)
  
  en_toks_copy,ar_toks_copy=list(en_toks),list(ar_toks)
  en_toks=[v.lower() for v in en_toks]

  if len(en_toks)>1:
    for h in heads:
      en_toks_copy[h]='<b><u>'+en_toks_copy[h]+'</u></b>'

  print(src)
  print(trg)
  results=[]

  for en0 in en_toks:
    #en0=en0.lower()
    #print(en0)
    retrieved_phrases=src_word_index_dict.get(en0,[])
    for en_phrase in retrieved_phrases:
      if en_phrase.isdigit(): continue
      en_phrase_toks=en_phrase.split(" ")
      found_en=list_in_list(en_phrase_toks,en_toks)
      if not found_en: continue
      #print(found_en)
      
      cur_corr=corr_dict[en_phrase]

      for ar_phrase,val in cur_corr.items():
        ar_phrase_toks=ar_phrase.split(" ")
        found_ar=list_in_list(ar_phrase_toks,ar_toks)
        if not found_ar: continue
        wt=src_trg_wt_dict.get((en_phrase,ar_phrase),0)
        combined_size=len(ar_phrase_toks)+len(en_phrase_toks)
        new_wt=wt*log(val)*log(combined_size)
        results.append((en_phrase,ar_phrase,found_en,found_ar,val,new_wt))
        #print(">>", en_phrase,ar_phrase,val,wt)
    #print('----')
    
  #print(i_,en_toks,ar_toks)
  results.sort(key=lambda x:-x[-1])
  used_src,used_trg=[],[]
  final_phrase_pairs=[]
  for r in results:
    en_phrase,ar_phrase,found_en,found_ar,val,new_wt=r
    if en_phrase in used_src: continue
    if ar_phrase in used_trg: continue
    used_src.append(en_phrase)
    used_trg.append(ar_phrase)
    final_phrase_pairs.append(r)
  used_xs=[]
  for ri,r in enumerate(final_phrase_pairs):
    en_phrase,ar_phrase,found_en,found_ar,val,new_wt=r
    #color = "#%06x" % random.randint(0, 0xFFFFFF)
    color = random_color()
    #background-color
    #cur_span='<span style="color:%s" class="ph%s">'%(color,ri)
    cur_span='<span style="background-color:%s" class="ph%s">'%(color,ri)
    exclude=False
    for x0,x1 in found_en:
      if x0 in used_xs or x1 in used_xs: exclude=True
      if not exclude:
        used_xs.extend(list(range(x0,x1+1)))
        en_toks_copy[x0]=cur_span+en_toks_copy[x0]
        en_toks_copy[x1]=en_toks_copy[x1]+'</span>'
    for y0,y1 in found_ar:
      if not exclude:
        ar_toks_copy[y0]=cur_span+ar_toks_copy[y0]
        ar_toks_copy[y1]=ar_toks_copy[y1]+'</span>'

    #en_toks_copy,ar_toks_copy
    #print(r)
  src_cell=" ".join(en_toks_copy)
  trg_cell=" ".join(ar_toks_copy)
  print(" ".join(en_toks_copy) )
  print(" ".join(ar_toks_copy))
  output_fopen.write('<tr><td>%s</td><td dir="rtl">%s</td></tr>'%(src_cell,trg_cell))


  print("=====")
  #if i_>20: break
fopen.close()
output_fopen.write("</table></body></html>")
output_fopen.close()


#print(color)

# for key,val in corr_dict.items():
#   print(key,val)

heads >>>>> [0]
C.
جيم-
<span style="background-color:#ECC6C6" class="ph0"><b><u>C</u></b></span> .
<span style="background-color:#ECC6C6" class="ph0">جيم</span> -
=====
heads >>>>> [2]
Regional mission cooperation
التعاون مع البعثات العاملة في المنطقة
<span style="background-color:#A9F5A3" class="ph1">Regional</span> mission <span style="background-color:#F3DCB0" class="ph0"><b><u>cooperation</u></b></span>
<span style="background-color:#A9F5A3" class="ph1"><span style="background-color:#F3DCB0" class="ph0">ال_تعاون</span> مع</span> ال_بعثات ال_عاملة في ال_منطقة
=====
heads >>>>> [2]
MINUSCA will seek to strengthen its inter-regional cooperation and economies of scale with missions in the region to enhance collaboration and analysis, where feasible.
ستسعى البعثة إلى تعزيز تعاونها المشترك مع البعثات الأخرى العاملة في المنطقة وما تحققه من وفورات الحجم لتعزيز التآزر والتحليل، حيثما أمكن ذلك.
MINUSCA <span style="background-color:#E8C5C9" class="ph2">will</span> <span style="background-co

In [ ]:
#Doing the actual word alignment
walign_obj.align_sent_words()

In [ ]:
import math
print(math.log(100))
print(math.log(2000))
print(math.log(20000))
print(math.log(200000))

print(math.log(15))
print(math.log(10))
print(math.log(5))
print(math.log(29))
print(math.log(22))

4.605170185988092
7.600902459542082
9.903487552536127
12.206072645530174
2.70805020110221
2.302585092994046
1.6094379124341003
3.367295829986474
3.091042453358316


In [ ]:
ignored_src_toks=["the","and","of","an"]
ignored_trg_toks=["وـ","لـ"] #we should also include ignoring prefixes and suffixes with tatweel
def index_src_trg(src_sents,trg_sents,ignore_punct=True,ignore_ar_pre_suf=False,ignored_src=[],ignored_trg=[]):
  src_fwd_index,trg_fwd_index=[],[]
  for s_i,st in enumerate(zip(src_sents,trg_sents)): 
    src_toks,trg_toks=st
    src_toks=[norm_tok(v) for v in src_toks]
    trg_toks=[norm_tok(v) for v in trg_toks]

    if ignore_punct:
      src_toks=[v for v in src_toks if not is_punct(v) and not v in ignored_src]
      trg_toks=[v for v in trg_toks if not is_punct(v) and not v in ignored_trg]
    else:
      src_toks=[v for v in src_toks if not v in ignored_src]
      trg_toks=[v for v in trg_toks if not v in ignored_trg]


    src_toks=[(v,s_i,t_i) for t_i,v in enumerate(src_toks)]
    trg_toks=[(v,s_i,t_i) for t_i,v in enumerate(trg_toks)]
    src_fwd_index.extend(src_toks)
    trg_fwd_index.extend(trg_toks)
  src_fwd_index.sort()
  trg_fwd_index.sort()
  src_grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(src_fwd_index,lambda x:x[0])]
  trg_grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(trg_fwd_index,lambda x:x[0])]

  src_inverted=dict(iter(src_grouped))
  trg_inverted=dict(iter(trg_grouped))  
  return src_inverted,trg_inverted

cur_src_inverted,cur_trg_inverted=index_src_trg(src_sents_tok,trg_sents_tok,ignored_src=ignored_src_toks, ignored_trg=ignored_trg_toks)

for a in list(cur_src_inverted.keys())[1000:1100]:
  print(a,cur_src_inverted[a])

for a in list(cur_trg_inverted.keys())[1000:1100]:
  print(a,cur_trg_inverted[a])



689 [(2892, 3), (7684, 11), (15542, 21)]
68th [(3969, 3), (6050, 2), (7675, 3), (7724, 2), (7771, 3), (7843, 3), (8079, 3)]
69 [(245, 0), (1178, 5), (3020, 1), (3020, 3), (3703, 17), (3973, 17), (4614, 0), (6054, 18), (7122, 13), (7480, 0), (7679, 17), (7728, 16), (7775, 17), (7848, 17), (8083, 17), (8399, 3), (8399, 6), (8611, 7), (11369, 0), (11701, 12), (11808, 4), (11875, 29), (11915, 17), (13968, 16), (14766, 0), (14824, 6), (14824, 11), (14824, 16), (14824, 21), (14824, 26), (14824, 31), (14824, 36), (14824, 41), (14825, 15), (15502, 3), (15675, 0), (16118, 9), (17157, 0), (17191, 14), (17621, 0), (18201, 6), (18473, 6), (18731, 0), (19391, 0), (19879, 0)]
690 [(8390, 2), (8390, 5), (8475, 8)]
691 [(7431, 15), (7459, 43), (15640, 8)]
692 [(5297, 7)]
693 [(5154, 3), (6060, 6), (15930, 14)]
694 [(8669, 10), (9421, 6), (12308, 4), (12574, 13), (12924, 24)]
696 [(5338, 4), (12585, 5), (13599, 13)]
698 [(623, 5), (804, 25), (924, 17), (3874, 10)]
69th [(3969, 4), (6050, 3), (7675, 4),

In [ ]:
for src0,trg0 in zip(src_sents_tok[:100],trg_sents_tok[:100]):
  print(src0)
  print(trg0)
  print("------")

['ORIGINAL', ':', 'ENGLISH', '/', 'SPANISH']
['ORIGINAL', ':', 'ENGLISH', '/', 'SPANISH']
------
['INTERNATIONAL', 'LAW', 'COMMISSION']
['لجنة', 'ال_قانون', 'ال_دولي']
------
['Forty', '-', 'fifth', 'session']
['ال_دورة', 'ال_خامسة', 'وـ', 'ال_أربعون']
------
['3', 'May', '-', '23', 'July', '1993']
['3', 'أيار', '/', 'مايو', '-', '23', 'تموز', '/', 'يوليه', '1993']
------
['COMMENTS', 'OF', 'GOVERNMENTS', 'ON', 'THE', 'REPORT', 'OF', 'THE', 'WORKING', 'GROUP']
['تعليقات', 'ال_حكومات', 'على', 'تقرير', 'ال_فريق', 'ال_عامل', 'ال_معني']
------
['ON', 'THE', 'QUESTION', 'OF', 'AN', 'INTERNATIONAL', 'CRIMINAL', 'JURISDICTION']
['بـ', 'مسألة', 'إنشاء', 'قضاء', 'جنائي', 'دولي']
------
['CONTENTS']
['ال_محتويات']
------
['Page']
['ال_صفحة']
------
['INTRODUCTION', '.', '2']
['مقدمة', '.', '2']
------
['Australia', '.', '3']
['استراليا', '.', '4']
------
['Denmark', '*', '.', '17']
['ال_دانمرك', '*', '.', '21']
------
['Finland', '*', '.', '17']
['ال_سويد', '*', '.', '21']
------
['Iceland', '*'

In [ ]:
#now extracting templates from UNTerm entries
for i,item in enumerate(unterm_list[:10]):
  en_words,ar_words=item
  print(en_words,ar_words)
  en_words_lower=[v.lower() for v in en_words]
  ar_words_no_al=[v.split("_")[-1] for v in ar_words]
  possible_sub_items_indexes=[]

  for en0 in en_words_lower:
    possible_sub_items_indexes.extend(pivot_dict.get(en0,[]))
    #print(en0, pivot_dict.get(en0,[]))
  found_sub_items=[]
  for ind in possible_sub_items_indexes:
    if ind==i: continue
    sub_en,sub_ar=unterm_list[ind]
    sub_en_lower=[v.lower() for v in sub_en]
    check_en=phrase_in_phrase(sub_en_lower,en_words_lower)
    if check_en:
      #print(ind, check_en, sub_en_lower)
      sub_ar_no_al=[v.split("_")[-1] for v in sub_ar]
      found_sub_items.append((ind, sub_en_lower,check_en,sub_ar_no_al))
  found_src_trg=[]
  for fs in found_sub_items:
    #print(fs)
    #cur_sub_ar=fs[-1]
    ind, sub_en_lower,check_en,sub_ar_no_al=fs
    check_ar=phrase_in_phrase(sub_ar_no_al,ar_words_no_al)
    if check_ar:found_src_trg.append((ind, sub_en_lower,check_en,sub_ar_no_al,check_ar))
  if found_src_trg:
    print(i,en_words)
    print(ar_words)
    found_src_trg.sort(key=lambda x:-len(x[1]))
    template_src,template_trg=list(en_words),list(ar_words)
    placeholder_i=0
    for fst in found_src_trg:
      ind, src_words,src_locs,trg_words,trg_locs=fst
      print(fst)

      for ii in range(len(src_locs)):
        
        placeholder_name="_x%s_"%placeholder_i
        placeholder_i+=1
        #print(placeholder_name,ii,src_locs)
        src_i,trg_i=src_locs[ii],trg_locs[ii]
        placeholder_src=[placeholder_name]*len(src_words)
        placeholder_trg=[placeholder_name]*len(trg_words)
        template_src[src_i:src_i+len(src_words)]=placeholder_src
        template_trg[trg_i:trg_i+len(trg_words)]=placeholder_trg
    template_src=[key for key,group in groupby(template_src)]
    template_trg=[key for key,group in groupby(template_trg)]
    print(template_src)
    print(template_trg)
    print("--------")
  


['United', 'Nations', 'Decade', 'against', 'Drug', 'Abuse'] ['عقد', 'ال_أمم', 'ال_متحدة', 'لـ', 'مكافحة', 'إساءة', 'استعمال', 'ال_مخدرات']
0 ['United', 'Nations', 'Decade', 'against', 'Drug', 'Abuse']
['عقد', 'ال_أمم', 'ال_متحدة', 'لـ', 'مكافحة', 'إساءة', 'استعمال', 'ال_مخدرات']
(20183, ['united', 'nations'], [0], ['أمم', 'متحدة'], [1])
['_x0_', 'Decade', 'against', 'Drug', 'Abuse']
['عقد', '_x0_', 'لـ', 'مكافحة', 'إساءة', 'استعمال', 'ال_مخدرات']
--------
['Agreement', 'between', 'the', 'United', 'Nations', 'and', 'the', 'World', 'Intellectual', 'Property', 'Organization'] ['ال_اتفاق', 'ال_معقود', 'بين', 'ال_أمم', 'ال_متحدة', 'وـ', 'ال_منظمة', 'ال_عالمية', 'لـ', 'ال_ملكية', 'ال_فكرية']
1 ['Agreement', 'between', 'the', 'United', 'Nations', 'and', 'the', 'World', 'Intellectual', 'Property', 'Organization']
['ال_اتفاق', 'ال_معقود', 'بين', 'ال_أمم', 'ال_متحدة', 'وـ', 'ال_منظمة', 'ال_عالمية', 'لـ', 'ال_ملكية', 'ال_فكرية']
(212, ['world', 'intellectual', 'property', 'organization'], [7], ['

# New Functions
In this section, we test and update utility functions

In [ ]:
#Now for the templating functions
big=["a","c","a","b","c","g","a","b","n","c","a","b","c","d"]
small=["a","b","c"]
repl="__x0__"

def make_template(big_list,small_list,repl_symbol):
  copy_big=[]
  last_i=0
  for i0,item in enumerate(big_list):
    if big_list[i0:i0+len(small_list)]==small_list: 
      copy_big.extend(big_list[last_i:i0])
      copy_big.append(repl_symbol)
      last_i=i0+len(small_list)
  copy_big.extend(big_list[last_i:])
  return copy_big

def match_template(template,sequence,include_last=False):
  pass
  

test_big="about other people and other people who like to talk to other people in the street"
test_big="about other people and other people"
test_big_toks=test_big.split()
test_small="other people"
test_small_toks=test_small.split()

test_template=make_template(test_big_toks,test_small_toks,"__X1__")
print(test_big)
print(test_template)

# copy_big=[]

# last_i=0
# for i0,item in enumerate(big):
#   if big[i0:i0+len(small)]==small: 
#     copy_big.extend(big[last_i:i0])
#     copy_big.append(repl)
#     last_i=i0+len(small)

# copy_big.extend(big[last_i:])
# print(big)
# print(copy_big)

about other people and other people
['about', '__X1__', 'and', '__X1__']


In [ ]:
#Testing Arabic word prefix/suffix analysis and splitting
sent="وهذه الالتزامات والالتماسات التي التزمت بها الدول فيما يخص التنمية والتاريخ والتراث والتزامات التزام التماساتها"
ar_words=tok(sent)
ar_words=tok_ar(ar_words,ar_counter_dict)
print(ar_words)
arw="والاتجار"
arw="الشرقين"
cur_candidates=get_pre_suf_candidates(arw)
sorted_candidates=sort_filter(cur_candidates,ar_counter_dict)
for a0 in sorted_candidates:
  print(a0)

['وـ', 'هذه', 'ال_التزامات', 'وـ', 'ال_التماسات', 'التي', 'التزمت', 'بها', 'ال_دول', 'فيما', 'يخص', 'ال_تنمية', 'وـ', 'ال_تاريخ', 'وـ', 'ال_تراث', 'وـ', 'التزامات', 'التزام', 'التماسات', 'ـها']
(('ال', 'شرقين', '', 'al-'), 1)
(('', 'الشرقين', '', '-'), 0)


In [ ]:
#Let's use this part to test the regex for certain elements we want to avoid splitting while tokenizing, such as websites, UN symbols
import re
sent="symbol A/71/241.add2 and A/RES/149/78 and http://www.hotmail.com/who-is $19.571 https://what-is-love.com/when"
re.findall("https?\:\/\/\S+",sent)

['http://www.hotmail.com/who-is', 'https://what-is-love.com/when']

In [ ]:
#testing the tokenization function, maybe it will need to be tested and maintained against new/unexpected data
from code_utils.general import *
sent="-I don't think it is a big (problem), but we'll see! In fact, many people can; including me, get really confused?"
sent="symbol A/71/241.add2 and A/RES/149/78 and http://www.hotmail.com/who-is $19.571"
tok(sent)

['symbol',
 'A/71/241.add2',
 'and',
 'A/RES/149/78',
 'and',
 'http://www.hotmail.com/who-is',
 '$',
 '19',
 '.',
 '571']

In [ ]:
#This is the tokenization function, if we update it here, we should also update it on Github
import re
def tok(txt,keep_urls=True,keep_un_symbols=True,keep_numbers=False): #this is a tokenization scheme to preserve the punctuation also, but it is sensetive to English clitics, instead of splitting isn't as ['isn',"'","t"], it splits ["is","n't"]
    replaced=[]
    if keep_urls: replaced.extend(re.findall("https?\:\/\/\S+",txt))
    if keep_un_symbols: replaced.extend(re.findall(r"[A-Z]+/\S+\d\b",txt))
    #if keep_numbers: replaced.extend(re.findall(r"[\d,\.]+",txt))
    repl_dict={}
    for i0,u0 in enumerate(replaced):
        key="__item%s__"%i0
        repl_dict[key]=u0
        txt=txt.replace(u0,key)

    #txt=txt.replace(u'\x01'," ")
    txt=txt.replace(u'\u2019',"'")
    
    txt=txt.replace("'s ","_s ")
    txt=txt.replace("'re ","_re ")
    txt=txt.replace("can't ","cann_t ")
    txt=txt.replace("cannot ","can not")
    txt=txt.replace("n't ","n_t ")
    txt=re.sub("(?u)(\W)",r" \1 ", txt)
    txt=txt.replace("_s ", " 's ")
    txt=txt.replace("_re "," 're ")
    txt=txt.replace("n_t "," n't ")
    for a,b in repl_dict.items():
      txt=txt.replace(a,b)
    
    out=re.split("\s+",txt)
    return [v for v in out if v]

sent="-I don't think it is a big (problem), but we'll see! In fact, many people can; including me, get really confused?"
sent="symbol A/71/241.add2 and A/RES/149/78  and A/C.123/1234. http://www.hotmail.com/who-is $19.571 S/198/490. (A/B.125/124) 1998/1999 A/1999/12"
#re.findall(r"[A-Z]+/\S+\d\b",sent)
tok(sent)


['symbol',
 'A/71/241.add2',
 'and',
 'A/RES/149/78',
 'and',
 'A/C.123/1234',
 '.',
 'http://www.hotmail.com/who-is',
 '$',
 '19',
 '.',
 '571',
 'S/198/490',
 '.',
 '(',
 'A/B.125/124',
 ')',
 '1998',
 '/',
 '1999',
 'A/1999/12']

In [99]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer=nlp.tokenizer.tokens_from_list
text="she went to school"
words=text.split()
doc = nlp(words)
for token in doc:	
  token_i = token.i+1
  if token.i==token.head.i: head_i=0
  else: head_i = token.head.i+1
  items=[token_i,token.text, token.lemma_, token.tag_, token.pos_, "_", head_i, token.dep_,"_","_"]
  print(items)

[1, 'she', '-PRON-', 'PRP', 'PRON', '_', 2, 'nsubj', '_', '_']
[2, 'went', 'go', 'VBD', 'VERB', '_', 0, 'ROOT', '_', '_']
[3, 'to', 'to', 'IN', 'ADP', '_', 2, 'prep', '_', '_']
[4, 'school', 'school', 'NN', 'NOUN', '_', 3, 'pobj', '_', '_']
